In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
# changed and escaped slashes in path, as i'm on windows
file_dir = "C:\\Users\\me\\Desktop\\0 berbooca\\github\\Movies-ETL\\"

### wikimedia

In [3]:
raw_file = f"{file_dir}wikipedia-movies.json"
with open(raw_file, mode='r') as file:
    wiki_movies_raw = json.load(file)

In [ ]:
# initial exploration, not needed later
print(f"wiki_movies_raw has {len(wiki_movies_raw)} records")
print(f"wiki_movies_raw is a {type(wiki_movies_raw)}")
wiki_movies_raw[3358]

In [ ]:
# initial exploration, not needed later
wiki_movies_df = pd.DataFrame(wiki_movies_raw)

In [ ]:
# initial exploration, not needed later
col_list = wiki_movies_df.columns.tolist()
print(f"this dataframe got {len(col_list)} columns - too many!")

In [4]:
# as suggested by module
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]

len(wiki_movies)
# lost 311+24 = 335

7076

In [5]:
wiki_movies_df = pd.DataFrame(wiki_movies)
col_list = wiki_movies_df.columns.tolist()
print(f"now we got {len(col_list)} columns, and here they are")
col_list

now we got 75 columns, and here they are


['url',
 'year',
 'imdb_link',
 'title',
 'Directed by',
 'Produced by',
 'Screenplay by',
 'Story by',
 'Based on',
 'Starring',
 'Narrated by',
 'Music by',
 'Cinematography',
 'Edited by',
 'Productioncompany ',
 'Distributed by',
 'Release date',
 'Running time',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Written by',
 'Genre',
 'Theme music composer',
 'Country of origin',
 'Original language(s)',
 'Producer(s)',
 'Editor(s)',
 'Production company(s)',
 'Original network',
 'Original release',
 'Productioncompanies ',
 'Executive producer(s)',
 'Production location(s)',
 'Distributor',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Composer(s)',
 'Created by',
 'Preceded by',
 'Adaptation by',
 'Suggested by',
 'Traditional',
 'Mandarin',
 'Released',
 'Recorded',
 'Venue',
 'Length',
 'Label',
 'Director',
 'Producer',
 'Also known as',
 'Animation by',
 'Color process',
 'Simplified',
 'French',
 'Screen story by',
 'Hangul',
 'Revised Romanization

In [ ]:
wiki_movies_df["Starring"]
# beware, beware: mutable entries in a df are passed-by-reference, even with .copy() !!

In [ ]:
wiki_cts = wiki_movies_df.count().sort_values(ascending=False)
wiki_cts_df = pd.DataFrame(wiki_cts)
wiki_cts_df.to_csv('wikicounts.csv')
wiki_cts.to_dict()

In [ ]:
# freak entry with no title, should be "Jacob"
jacob = wiki_movies_df[wiki_movies_df["title"].isna()].T
jacob[jacob[996].isna() == False]

In [ ]:
# result of my investigation, matches what module gives :)
other_title_columns = ['Traditional', 'Simplified', 'Hepburn', 'Japanese', 'Mandarin',
 'McCune–Reischauer', 'Revised Romanization', 'Hangul', 'Arabic', 'Romanized', 'Russian',
 'Yiddish', 'Cantonese', 'Chinese', 'Hebrew', 'Original title', 'French', 'Also known as',
 'Polish', "Literally"]

In [ ]:
wiki_movies_df["has alt title"] = False
for alt_title in other_title_columns:
    wiki_movies_df.loc[wiki_movies_df[alt_title].notnull(), ["has alt title"]] = True
wiki_movies_df["has alt title"].sum()

In [ ]:
wiki_movies_df["Literally"].value_counts()

In [ ]:
# ain't gonna redefine the balue of the input variable, that'll confusel me!
def clean_movie(movie):
    cp_movie = dict(movie) #create a non-destructive copy
    alt_titles_dict = {}
    for kee in other_title_columns:
        if kee in cp_movie:
            alt_titles_dict[kee] = cp_movie[kee]
            cp_movie.pop(kee)
    if len(alt_titles_dict) > 0:
        cp_movie["other titles"] = alt_titles_dict
    return cp_movie

In [ ]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]

In [ ]:
wiki_movies_df = pd.DataFrame(clean_movies)
wiki_cts = wiki_movies_df.count().sort_values(ascending=False)
wiki_cts_df = pd.DataFrame(wiki_cts)
print(len(wiki_cts_df))
wiki_cts.to_dict()

### kaggle

In [ ]:
kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}ratings.csv')

In [ ]:
ratings.sample(3)

In [ ]:
kaggle_metadata.sample(3)

In [ ]:
kaggle_metadata.count()

In [ ]:
kaggle_metadata.dtypes

In [ ]:
kaggle_metadata[kaggle_metadata["vote_count"].isna()]